In [1]:
"""
Project for Olin College, Data Science, Spring 2017.

Data from:
https://www.nass.usda.gov/Surveys/Guide_to_NASS_Surveys/Bee_and_Honey/
http://usda.mannlib.cornell.edu/MannUsda/viewDocumentInfo.do?documentID=1191


unzip the csv zip files with

$  for VARIABLE in 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016; 
$  do   unzip *$VARIABLE.zip -d $VARIABLE; 
$  done

or pull them from my git.
"""

import numpy as np
import pandas
import os
import string

In [2]:
def get_all_csvs(csv_years=[2002+i for i in range(15)], csv_path="data/csvs/"):
    '''
    Returns a list of all csv files in the (hard-coded) year-directory combinations.
    '''
    all_csvs = []
    for i in csv_years:
        this_path = csv_path + str(i) + "/"
        these_files = os.listdir(this_path)
        these_csvs = [(this_path + i) for i in these_files if i[-4:] == ".csv"]
        all_csvs.extend(these_csvs)
    return all_csvs

In [37]:
class Csv_obj(object):
    def __init__(self, filename):
        self.name = filename
        year_start = filename.find("20")
        self.year = int(filename[year_start:year_start+4])
        self.data_type = filename[(filename.find("_") + 1) : (filename.find(".csv"))]
        # 001: honey type/color/grade to quantity, price for that year
        #   color class : price (cents per pound), 
        #   Co-op/private : Retail : All
        # 002: State, number of colonies (1000s), 
        #   Yield (per colony), Yield (total, 1000 lbs), stocks (1klbs),
        #   Average price per pound (cents),
        #   total value of production.
        # 003: 
        temp_df = sanitize_then_load_csv(filename)
        self.df = self.clean_df(temp_df)

    
    def __str__(self):
        return str(self.year) + ":" + self.data_type
    
    def clean_df(self, target):
        if self.data_type[-3:] == "002":
            col_names = ["state","colonies","yield","production","stocks","prices","totalval"]
                        # abbr,   1000s      lbs/col  1000 lbs    1000 lbs  cents    $1000s   
            final = target.drop(target.columns[0],1).drop(target.columns[1],1).dropna()
            final.columns = col_names
            final = final[final.state != "State"]
            return final
        else:
            return target # todo: clean; right now 002 is all we care about though.
    
    def isnull(self):
        if self.df is None:
            return True
        else:
            return False

In [38]:
def sanitize_then_load_csv(filename, min_col_length=9, max_col_length = None):
    """
    Gets around the metadata-y columns by scanning for the length of .split(',').
    TODO: make it ignore the "format"/grammatical commas which occur within a single entry in the csv.
    """
    f = open(filename)
    lines_to_skip = []
    for i, line in enumerate(f.readlines()):
        l_split = line.split(',') # will also split internal commas in strings. todo: fix. low priority.
        if len(l_split) < min_col_length:
            lines_to_skip.append(i)
        elif max_col_length != None:
            if len(l_split) > max_col_length:
                lines_to_skip.append(i)
    try:
        df = pandas.read_csv(filename, skiprows = lines_to_skip)
    except Exception as err:
        print "Cannot read: ", filename, "\t ", str(err)[str(err).find("Expected"):]
        df = None
    return df

In [39]:
all_csvs = get_all_csvs()

In [40]:
csv_objs = [Csv_obj(i) for i in all_csvs]

Cannot read:  data/csvs/2008/hony_all.csv 	  Expected 9 fields in line 155, saw 11

Cannot read:  data/csvs/2009/hony_all.csv 	  Expected 9 fields in line 155, saw 11

Cannot read:  data/csvs/2010/hony_all.csv 	  Expected 9 fields in line 155, saw 11

Cannot read:  data/csvs/2011/hony_all_tables.csv 	  Expected 9 fields in line 153, saw 11

Cannot read:  data/csvs/2012/hony_all_tables.csv 	  Expected 9 fields in line 152, saw 10

Cannot read:  data/csvs/2013/hony_all_tables.csv 	  Expected 9 fields in line 152, saw 10

Cannot read:  data/csvs/2014/hony_all_tables.csv 	  Expected 9 fields in line 151, saw 10

Cannot read:  data/csvs/2015/hony_all_tables.csv 	  Expected 9 fields in line 151, saw 10

Cannot read:  data/csvs/2016/hony_p00a_t005.csv 	  e
Cannot read:  data/csvs/2016/hony_all_tables.csv 	  Expected 9 fields in line 166, saw 10



In [41]:
state_to_vals_dict = {}
        

SyntaxError: invalid syntax (<ipython-input-41-396d1ebc1840>, line 6)

In [45]:
# for i in csv_objs:
#     print i
test = csv_objs[3].df
relevants = [i for i in csv_objs if i.]

In [46]:
test

,state,colonies,yield,production,stocks,prices,totalval
5,AL,16,78,1248,187,59,736
6,AZ,40,59,2360,1322,73,1723
7,AR,55,93,5115,3529,57,2916
8,CA,440,70,30800,11396,59,18172
9,CO,29,60,1740,957,62,1079
10,FL,232,105,24360,2923,54,13154
11,GA,55,57,3135,376,63,1975
12,HI,7,112,784,125,82,643
13,ID,100,47,4700,2679,52,2444
14,IL,8,61,488,146,117,571
